In [ ]:
import matplotlib.pyplot as plt
import re
from matplotlib import rc
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from numpy.polynomial.polynomial import Polynomial
# from sklearn.metrics import r2_score
from tqdm import tqdm
from scipy.ndimage.filters import gaussian_filter1d

In [ ]:
def smooth(data, window_size):
    """Smooth the data using a window with requested size."""
    window = np.ones(window_size) / window_size
    return np.convolve(data, window, 'same')

def smooth_gaussian(data, sigma):
    """Smooth the data using a window with requested size."""
    return gaussian_filter1d(data, sigma=sigma)

# Balanced server

In [ ]:
fontsize = 28
legend_fontsize = 22
fontsize = 36
legend_fontsize = 32
plt.rcParams.update({'font.size': fontsize})
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
linewidth = 4
#####
algorithms = ["optimal", "greedy", "balanced"]
colors = {"optimal": '#50C878', "greedy": '#CCAC00', "balanced" : '#c74c18'}
algo_names = {"optimal": 'Synthesized', "greedy": 'Greedy', "balanced" : 'Balanced'}
linestyles = {"optimal": 'solid', "greedy": 'dashed', "balanced" : 'dashdot'}
#####

In [ ]:
algorithms = ['greedy', 'balanced', 'optimal']

In [ ]:
df = pd.read_csv(f'collected_data/output_balanced_server_T1000_X20_pa03_pb075_B5.txt', skiprows = 4 )
df = pd.read_csv(f'collected_data/output_balanced_server_T100_X20_pa03_pb075_B5.txt', skiprows = 4 )
df = pd.read_csv(f'collected_data/output_balanced_server_T1000_X20_pa51_pb049_B5.txt', skiprows = 4 )

In [ ]:
for algo in algorithms:
    df[f"b_{algo}"] = df.apply(lambda row: row[f"b_{algo}"] * -1 if row[f"g_{algo}"] < 0.5 else row[f"b_{algo}"], axis=1)

In [ ]:
b_mean = {}
b_std = {}
b_min = {}
b_max = {}

rewards = {}

for algo in algorithms:
    b_mean[algo] = np.array(df[['t', f'b_{algo}']].groupby("t").mean()[f'b_{algo}'])
    b_std[algo] = np.array(df[['t', f'b_{algo}']].groupby("t").std()[f'b_{algo}'])
    b_min[algo] = np.array(df[['t', f'b_{algo}']].groupby("t").min()[f'b_{algo}'])
    b_max[algo] = np.array(df[['t', f'b_{algo}']].groupby("t").max()[f'b_{algo}'])
    rewards[algo] = df[df.t == df.t.max()][[f'reward_{algo}', 'sim_no']][f"reward_{algo}"].mean()


In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(9,5))

# plt.figure(figsize=(10, 6))

Times = df.t.unique()

sigma = 10

for agent in algorithms:
    
    
    smoothed_min = smooth_gaussian(b_min[agent], sigma)
    smoothed_max = smooth_gaussian(b_max[agent], sigma)
    
    
    plt.plot(Times, smoothed_min,
             label = algo_names[agent], color = colors[agent], 
             linewidth=linewidth, linestyle = linestyles[agent])
    plt.plot(Times, smoothed_max,
             label = None, color = colors[agent], 
             linewidth=linewidth, linestyle = linestyles[agent])




for spine in ['right', 'top']:
    ax.spines[spine].set_visible(False)


eps = 8
plt.axhline(eps, linestyle='dashed', label=None, xmin = 0.05, xmax = 0.96)
plt.axhline(-eps, linestyle='dashed', label=None, xmin = 0.05, xmax = 0.96)
plt.fill_between([Times[0], Times[-1]+10], [-eps, -eps], [eps, eps], color = 'blue', alpha = 0.05 )


plt.text(75, -4.96, "Target", fontsize = legend_fontsize-6)

# Add labels and title

plt.xlabel('Time steps')
plt.ylabel('Imbalance')

# plt.legend()


# Show the plot
plt.savefig('results_images/balanced_server_simulation.pdf', bbox_inches='tight')
plt.show()

# Maximally responsive server

In [ ]:
fontsize = 36
legend_fontsize = 34
plt.rcParams.update({'font.size': fontsize})
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
linewidth = 4
#####
colors = {0: '#50C878', 5: '#CCAC00', 10 : '#c74c18', 15: 'blue', 20: 'green', 25: 'red'}
colors = {0: '#3498DB', 5: '#E74C3C', 15 : '#F1C40F', 25: '#2ECC71'}
linestyles = {0: 'solid', 5: 'dashed', 15: 'dotted', 25 : 'dashdot'}
#####

In [ ]:
budgets = [5, 10, 15, 20, 25]
budgets = [5, 15, 25]
all_df = {}
for budget in budgets:
    df = pd.read_csv(f'collected_data/output_max_response_T100_X6_pa03_pb075_B{budget}.txt', skiprows = 4 )
    df["used_budget"] = df["cost_greedy"] - df["cost_optimal"]
    all_df[budget] = df

In [ ]:
time = np.sort(all_df[budgets[0]].t.unique())

In [ ]:
lost_revenue = {}
lost_revenue[0] = 0

for budget in budgets:

    df = all_df[budget][["t", "cost_greedy", "cost_optimal"]]
    lost_revenue[budget] = 100*(df[df.t == 99].cost_greedy.mean() - df[df.t == 99].cost_optimal.mean())/(df[df.t == 99].cost_greedy.mean())

In [ ]:
m_std = {}
m_mean = {}



mean_greedy = np.array(all_df[budgets[0]][["t", "m_greedy"]].groupby("t").mean().m_greedy)
std_greedy = np.array(all_df[budgets[0]][["t", "m_greedy"]].groupby("t").std().m_greedy)

for budget in budgets:
    m_mean[budget] = np.array(all_df[budget][["t", "m_optimal"]].groupby("t").mean().m_optimal)
    m_std[budget] = np.array(all_df[budget][["t", "m_optimal"]].groupby("t").std().m_optimal)
m_mean[0] = mean_greedy
m_std[0] = std_greedy
budgets.append(0)

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(8.5,5))

# plt.figure(figsize=(10, 6))

window_size = 10


for budget in budgets:
    plt.plot(time, m_mean[budget], label=f"{budget} - {lost_revenue[budget]:.2f} \% lost", 
             color = colors[budget], linestyle = linestyles[budget], linewidth = linewidth)
    plt.fill_between(time, m_mean[budget] - m_std[budget], m_mean[budget] + m_std[budget], 
                     color=colors[budget], alpha=0.15)



for spine in ['right', 'top']:
    ax.spines[spine].set_visible(False)

# 
# plt.text(75, -0.06, "Target", fontsize = legend_fontsize-6)

# Add labels and title

plt.xlabel('Time steps')
plt.ylabel('Max. wait time')

# plt.legend(fontsize=legend_fontsize)




# Show the plot
plt.savefig('results_images/max_response_time.pdf', bbox_inches='tight')
plt.show()

# Established Clientele

In [ ]:
fontsize = 28
legend_fontsize = 22
fontsize = 36
legend_fontsize = 32
plt.rcParams.update({'font.size': fontsize})
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
linewidth = 4

#####
agents = ["optimal", "threshold"]
# budgetlist = [50, 100, 150, 200, 250, 300, 450]
budgetlist = [50, 150, 250, 350]

colors = {
    50: '#dc3320',
    150: '#0d7a1e',
    250: '#3357FF',
    350: '#c215be',
    450: '#99872f'
}

linestyles = {
    50: 'solid',
    150: '--',
    250: '-.',
    350: ':',
    450: 'solid'
}

# colors = {"optimal": '#50C878', "threshold": '#CCAC00'}
algo_names = {"optimal": 'Optimal', "threshold": 'Threshold'}
# linestyles = {"optimal": 'solid', "threshold": 'dashed'}
#####

In [ ]:
toplot = {}
for budget in budgetlist:
    all_data_df = pd.read_csv(f'collected_data/established_clientele_simulations/established_clientele_input_N100_Dcenteredbino_l-5_u4_n6_k1_T500_B{budget}_S10/data/all_data.csv')
    # data_df = all_data_df[['simulation', 'time', 'variable', 'value', 'group', 'agent']]
    data_df = all_data_df[(all_data_df.variable == "threshold") & (all_data_df.agent == "optimal")]
    # Use groupby and pivot to reshape the data
    data_df = data_df[['simulation', 'time', 'agent', 'variable', 'value']]
    threshold_data = data_df.pivot_table(index=['simulation', 'time'], values='value').unstack()
    threshold_data.columns = [col[1] for col in threshold_data.columns]
    threshold_data.loc['mean', :] = threshold_data.mean(axis=0)
    threshold_data.loc['std', :] = threshold_data.std(axis=0)
    toplot[budget] = {}
    toplot[budget]['mean'] = np.array(threshold_data.loc['mean', :])
    toplot[budget]['std'] = np.array(threshold_data.loc['std', :])
    

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(8, 5))

Times = data_df.time.unique()
window_size = 15  # Adjust the window size as needed for smoothing

for budget in budgetlist:
    # Smooth the data
    smoothed_mean = smooth(toplot[budget]['mean'], window_size)
    smoothed_std = smooth(toplot[budget]['std'], window_size)

    # Plot the smoothed data
    plt.plot(Times, smoothed_mean,
             label=f"Budget = {budget}", color=colors[budget], linestyle=linestyles[budget],
             linewidth=linewidth, alpha=0.8)
    plt.fill_between(Times, smoothed_mean - smoothed_std,
                     smoothed_mean + smoothed_std, color=colors[budget], alpha=0.2)

plt.axhline(0, linestyle='dashed', label=None, linewidth=0.5)

for spine in ['right', 'top']:
    ax.spines[spine].set_visible(False)

# Add labels and title
plt.xlabel('Time steps')
# plt.ylabel('Credit score threshold')
plt.ylabel('Price threshold')
# plt.legend(fontsize=legend_fontsize, loc=[0.1, 0.08], ncol=2, columnspacing=0.4)

# Show the plot
plt.savefig('results_images/established_clientele_simulation_threshold.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(8,5))

# plt.figure(figsize=(10, 6))

Times = data_df.time.unique()
for budget in budgetlist:
    plt.plot(Times, toplot[budget]['mean'],
             label = f"Budget = {budget}", color = colors[budget], linestyle = linestyles[budget],
             linewidth=linewidth,alpha=0.8)
    plt.fill_between(Times, toplot[budget]['mean'] - toplot[budget]['std'],
                    toplot[budget]['mean'] + toplot[budget]['std'], color = colors[budget], alpha = 0.2)

plt.axhline(0, linestyle='dashed', label=None, linewidth = 0.5)
    
for spine in ['right', 'top']:
    ax.spines[spine].set_visible(False)

# Add labels and title

plt.xlabel('Time steps')
plt.ylabel('Threshold')

plt.legend(fontsize=legend_fontsize, loc=[0.1, 0.08], ncol=2, columnspacing=0.4)

# Show the plot
plt.savefig('results_images/established_clientele_simulation_threshold.pdf', bbox_inches='tight')
plt.show()

    